# Integrate biogeographic traits with a geographic map

To enable microbiome data analysis in conjunction with the metadata extracted from a geographic map, we also implemented several functions to merge microbial traits with a `SpatialPolygonsDataFrame` or extract a new metadata from a `SpatialPolygonsDataFrame`. By using these functions, we can perform the statistical comparing based on administrative areas or grids. 

Here we need three R packages for this section of [microgeo](https://github.com/ChaonanLi/microgeo) R package tutorial. Just run the following codes to import them into R environment.

In [1]:
suppressMessages(require("magrittr")) 
require("ggplot2")  %>% suppressMessages()
require("microgeo") %>% suppressMessages()

If the Chinese characters cannot be displayed correctly, please run the following codes to set locale to `UTF-8`:

In [2]:
prev_locale <- Sys.setlocale("LC_CTYPE", "C.UTF-8") 

We need a standard microgeo dataset for the presentations in the section of tutorial.

In [3]:
# Use the map downloaded from DataV.GeoAtlas
data(qtp)
map <- read_aliyun_map(adcode = c(540000, 630000, 510000)) %>% suppressMessages() 
dataset.dts.aliyun <- create_dataset(mat = qtp$asv, ant = qtp$tax, met = qtp$met, map = map,
                                     phy = qtp$tre, env = qtp$env, lon = "longitude", lat = "latitude")
dataset.dts.aliyun %<>% rarefy_count_table()
dataset.dts.aliyun %<>% tidy_dataset()
dataset.dts.aliyun %<>% calc_alpha_div(measures = c("observed", "shannon")) 
dataset.dts.aliyun %<>% calc_beta_div(measures = c("bray", "jaccard")) 
dataset.dts.aliyun %>% show_dataset()

ℹ [2024-01-14 11:37:37] INFO ==> all samples fall within the map area!

ℹ [2024-01-14 11:37:37] INFO ==> dataset has been created successfully!

ℹ [2024-01-14 11:37:37] INFO ==> use `object %>% show_dataset()` to check the summary of dataset.

ℹ [2024-01-14 11:37:41] INFO ==> the ASV/gene abundance table has been rarefied with a sub-sample depth of 5310

✔ [2024-01-14 11:37:45] SAVE ==> results have been saved to: object$div$alpha

✔ [2024-01-14 11:38:26] SAVE ==> results have been saved to: object$div$beta



── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


Now, let's go through each of these functions and see how they are used.

## 1. Merge a `data.frame` with a map

Firstly, we check the `data.frame` of alpha diversity indices, and the `SpatialPolygonsDataFrame`.

In [4]:
# Check the data.frame of alpha diversity indices 
head(dataset.dts.aliyun$div$alpha)

,observed,shannon
,<dbl>,<dbl>
s1,998,6.352697
s2,918,6.192404
s3,853,6.114390
s4,1041,6.295121
s5,880,6.167230
s6,987,6.240516


In [5]:
# Check the `SpatialPolygonsDataFrame`
head(dataset.dts.aliyun$map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,DataV.GeoAtlas,microgeo,西藏自治区,88.38828,31.56375
2,DataV.GeoAtlas,microgeo,青海省,96.04353,35.72640
3,DataV.GeoAtlas,microgeo,四川省,102.69345,30.67454


In [6]:
# Change the names of Polygons
dataset.dts.aliyun$map@data$NAME <- c("Tibet", "Qinghai", "Sichuan") 
head(dataset.dts.aliyun$map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454


Then, we merge the `data.frame` of alpha diversity indices with a `SpatialPolygonsDataFrame`.

In [7]:
# Merge data to a `SpatialPolygonsDataFrame`
common.map.mean4df <- merge_dfs_to_map(map = dataset.dts.aliyun$map, dat = dataset.dts.aliyun$div$alpha, 
                                       met = dataset.dts.aliyun$met, med = 'mean')
head(common.map.mean4df@data[,1:12])
# Now, you can visualize the microbial traits (alpha diversity indices) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,observed_mean,shannon_mean,observed_sd,shannon_sd,observed_se,shannon_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375,664.0611,5.847912,238.2900,0.4771864,10.40975,0.02084599,524
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640,648.1267,5.837465,246.1559,0.5373178,11.70844,0.02555760,442
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454,705.9568,5.975145,197.7584,0.3683702,11.86076,0.02209338,278


We also can merge the `data.frame` of alpha diversity indices with a gridded `SpatialPolygonsDataFrame`. 

In [8]:
# Grid the map [`SpatialPolygonsDataFrame`]
gridded.map <- grid_map(map = dataset.dts.aliyun$map, res = 1.5) %>% suppressMessages
head(gridded.map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Gridded.Map,microgeo,1,83.74702,29.73742
2,Gridded.Map,microgeo,2,85.46302,28.50944
3,Gridded.Map,microgeo,3,86.67299,28.33105
4,Gridded.Map,microgeo,4,89.49169,28.25211
5,Gridded.Map,microgeo,5,88.12468,28.29693
6,Gridded.Map,microgeo,6,85.14919,29.46224


In [9]:
# Merge data to a gridded map
gridded.map.mean4df <- merge_dfs_to_map(map = gridded.map, dat = dataset.dts.aliyun$div$alpha, 
                                        met = dataset.dts.aliyun$met, med = 'mean')
head(gridded.map.mean4df@data[,1:12])
# Now, you can visualize the microbial traits (alpha diversity indices) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,observed_mean,shannon_mean,observed_sd,shannon_sd,observed_se,shannon_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,Gridded.Map,microgeo,1,83.74702,29.73742,592.5385,5.806529,146.9425,0.2260891,40.75452,0.06270583,13
2,Gridded.Map,microgeo,2,85.46302,28.50944,698.3333,5.943652,200.6024,0.2889430,115.81786,0.16682133,3
3,Gridded.Map,microgeo,3,86.67299,28.33105,551.3077,5.773239,145.2425,0.2757119,40.28303,0.07646873,13
4,Gridded.Map,microgeo,4,89.49169,28.25211,660.5000,5.771473,306.0485,0.5754502,153.02424,0.28772508,4
5,Gridded.Map,microgeo,5,88.12468,28.29693,472.1333,5.420437,151.1290,0.4048125,39.02135,0.10452215,15
6,Gridded.Map,microgeo,6,85.14919,29.46224,558.1667,5.757654,148.5140,0.2296232,30.31529,0.04687163,24


## 2. Merge a `distance matrix` with a map

Firstly, we check the distance `matrix`, and the `SpatialPolygonsDataFrame`.

In [10]:
# Check the distance matrix 
dataset.dts.aliyun$div$beta$bray[1:5, 1:5]

,s1,s2,s3,s4,s5
s1,0.0000000,0.4856874,0.5657250,0.4962335,0.4001883
s2,0.4856874,0.0000000,0.4702448,0.4229755,0.4099812
s3,0.5657250,0.4702448,0.0000000,0.3937853,0.4824859
s4,0.4962335,0.4229755,0.3937853,0.0000000,0.4293785
s5,0.4001883,0.4099812,0.4824859,0.4293785,0.0000000


In [11]:
# Check the `SpatialPolygonsDataFrame`
head(dataset.dts.aliyun$map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454


Then, we merge the distance `matrix` with a `SpatialPolygonsDataFrame`. 

In [12]:
# Merge distance matrix to a common map
common.map.mean4mx <- merge_mtx_to_map(map = dataset.dts.aliyun$map, dat = dataset.dts.aliyun$div$beta$bray, 
                                        met = dataset.dts.aliyun$met, var = 'bray', med = 'mean')
head(common.map.mean4mx@data[,1:9])
# Now, you can visualize the microbial traits (beta diversity distance matrix) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,bray_mean,bray_sd,bray_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375,0.8129943,0.1043087,0.0002817857,524
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640,0.7970325,0.1224428,0.0003922093,442
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454,0.7212848,0.1392175,0.0007094905,278


We also can merge a distance `matrix` with a gridded `SpatialPolygonsDataFrame`.

In [13]:
# Grid the map 
gridded.map <- grid_map(map = dataset.dts.aliyun$map, res = 1.5) %>% suppressMessages
head(gridded.map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Gridded.Map,microgeo,1,83.74702,29.73742
2,Gridded.Map,microgeo,2,85.46302,28.50944
3,Gridded.Map,microgeo,3,86.67299,28.33105
4,Gridded.Map,microgeo,4,89.49169,28.25211
5,Gridded.Map,microgeo,5,88.12468,28.29693
6,Gridded.Map,microgeo,6,85.14919,29.46224


In [14]:
# Merge distance matrix to a gridded map
gridded.map.mean4mx <- merge_mtx_to_map(map = gridded.map, dat = dataset.dts.aliyun$div$beta$bray, 
                                        met = dataset.dts.aliyun$met, var = 'bray', med = 'mean')
head(gridded.map.mean4mx@data[,1:9])
# Now, you can visualize the microbial traits (beta diversity distance matrix) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,bray_mean,bray_sd,bray_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,Gridded.Map,microgeo,1,83.74702,29.73742,0.6617847,0.09122993,0.010329755,13
2,Gridded.Map,microgeo,2,85.46302,28.50944,0.6444444,0.09513602,0.054926805,3
3,Gridded.Map,microgeo,3,86.67299,28.33105,0.7320682,0.13070514,0.014799443,13
4,Gridded.Map,microgeo,4,89.49169,28.25211,0.7667294,0.23362341,0.095376358,4
5,Gridded.Map,microgeo,5,88.12468,28.29693,0.7750265,0.13875375,0.013540980,15
6,Gridded.Map,microgeo,6,85.14919,29.46224,0.7700551,0.10876034,0.006546603,24


## 3. Extract metadata table from a map

In [15]:
# Extract metadata from a common map
# Rownames are sample IDs
# This new matadata table can be used for subsequent statistical analysis
metadata <- dataset.dts.aliyun$map %>% extract_metadata_from_map(met = dataset.dts.aliyun$met)
head(metadata)

,longitude,latitude,NAME,TYPE,FMTS,X.CENTER,Y.CENTER
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
s1,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s2,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s3,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s4,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s5,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s6,98.20639,33.10280,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454


In [16]:
# Extract metadata from a common map with additional data
# Rownames are sample IDs
# This new matadata table can be used for subsequent statistical analysis 
metadata.from.c.df <- common.map.mean4df %>% extract_metadata_from_map(met = dataset.dts.aliyun$met)
head(metadata.from.c.df)

,longitude,latitude,NAME,TYPE,FMTS,X.CENTER,Y.CENTER,observed_mean,shannon_mean,observed_sd,shannon_sd,observed_se,shannon_se,sample.num
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
s1,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,705.9568,5.975145,197.7584,0.3683702,11.86076,0.02209338,278
s2,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,705.9568,5.975145,197.7584,0.3683702,11.86076,0.02209338,278
s3,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,705.9568,5.975145,197.7584,0.3683702,11.86076,0.02209338,278
s4,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,705.9568,5.975145,197.7584,0.3683702,11.86076,0.02209338,278
s5,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,705.9568,5.975145,197.7584,0.3683702,11.86076,0.02209338,278
s6,98.20639,33.10280,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,705.9568,5.975145,197.7584,0.3683702,11.86076,0.02209338,278


In [17]:
# Extract metadata from a gridded map
# Rownames are sample IDs
# This new matadata table can be used for subsequent statistical analysis
metadata.from.g.mx <- gridded.map.mean4mx %>% extract_metadata_from_map(met = dataset.dts.aliyun$met)
head(metadata.from.g.mx)

,longitude,latitude,NAME,TYPE,FMTS,X.CENTER,Y.CENTER,bray_mean,bray_sd,bray_se,sample.num
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
s1,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6556815,0.1582218,0.00502862,45
s2,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6556815,0.1582218,0.00502862,45
s3,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6556815,0.1582218,0.00502862,45
s4,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6556815,0.1582218,0.00502862,45
s5,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6556815,0.1582218,0.00502862,45
s6,98.20639,33.10280,138,Gridded.Map,microgeo,98.70691,32.50087,0.6556815,0.1582218,0.00502862,45
